In [15]:
import torch
from safetensors import safe_open
from scipy.linalg import svd
import numpy as np

# Load the LoRA tensors from .safetensors files
with safe_open("lora1.safetensors", framework="pt", device="cpu") as f:
    lora1_tensors = {}
    for k in f.keys():
        lora1_tensors[k] = f.get_tensor(k)

# Print the available keys
print("Keys in lora1_tensors:")
for key in lora1_tensors.keys():
    print(key)

Keys in lora1_tensors:
lora_te_text_model_encoder_layers_0_mlp_fc1.alpha
lora_te_text_model_encoder_layers_0_mlp_fc1.lora_down.weight
lora_te_text_model_encoder_layers_0_mlp_fc1.lora_up.weight
lora_te_text_model_encoder_layers_0_mlp_fc2.alpha
lora_te_text_model_encoder_layers_0_mlp_fc2.lora_down.weight
lora_te_text_model_encoder_layers_0_mlp_fc2.lora_up.weight
lora_te_text_model_encoder_layers_0_self_attn_k_proj.alpha
lora_te_text_model_encoder_layers_0_self_attn_k_proj.lora_down.weight
lora_te_text_model_encoder_layers_0_self_attn_k_proj.lora_up.weight
lora_te_text_model_encoder_layers_0_self_attn_out_proj.alpha
lora_te_text_model_encoder_layers_0_self_attn_out_proj.lora_down.weight
lora_te_text_model_encoder_layers_0_self_attn_out_proj.lora_up.weight
lora_te_text_model_encoder_layers_0_self_attn_q_proj.alpha
lora_te_text_model_encoder_layers_0_self_attn_q_proj.lora_down.weight
lora_te_text_model_encoder_layers_0_self_attn_q_proj.lora_up.weight
lora_te_text_model_encoder_layers_0_self

In [17]:
with safe_open("lora2.safetensors", framework="pt", device="cpu") as f:
    lora2_tensors = {}
    for k in f.keys():
        lora2_tensors[k] = f.get_tensor(k)

print("\nKeys in lora2_tensors:")
for key in lora2_tensors.keys():
    print(key)


Keys in lora2_tensors:
lora_te_text_model_encoder_layers_0_mlp_fc1.alpha
lora_te_text_model_encoder_layers_0_mlp_fc1.lora_down.weight
lora_te_text_model_encoder_layers_0_mlp_fc1.lora_up.weight
lora_te_text_model_encoder_layers_0_mlp_fc2.alpha
lora_te_text_model_encoder_layers_0_mlp_fc2.lora_down.weight
lora_te_text_model_encoder_layers_0_mlp_fc2.lora_up.weight
lora_te_text_model_encoder_layers_0_self_attn_k_proj.alpha
lora_te_text_model_encoder_layers_0_self_attn_k_proj.lora_down.weight
lora_te_text_model_encoder_layers_0_self_attn_k_proj.lora_up.weight
lora_te_text_model_encoder_layers_0_self_attn_out_proj.alpha
lora_te_text_model_encoder_layers_0_self_attn_out_proj.lora_down.weight
lora_te_text_model_encoder_layers_0_self_attn_out_proj.lora_up.weight
lora_te_text_model_encoder_layers_0_self_attn_q_proj.alpha
lora_te_text_model_encoder_layers_0_self_attn_q_proj.lora_down.weight
lora_te_text_model_encoder_layers_0_self_attn_q_proj.lora_up.weight
lora_te_text_model_encoder_layers_0_sel

Now, in the paper on LoRAs, the following concept is introduced: The subspace similarity measure is a way of measuring the similarity between the subspaces spanned by the top singular vectors of two low-rank adaptation matrices, $A_{r=8}$ and $A_{r=64}$, from the same pre-trained model. Here's how it's done:

First, you perform a singular value decomposition (SVD) on each of these matrices to obtain their right-singular unitary matrices, denoted $U_{A_{r=8}}$ and $U_{A_{r=64}}$.

The goal is then to quantify how much of the subspace spanned by the top $i$ singular vectors in $U_{A_{r=8}}$ is contained in the subspace spanned by the top $j$ singular vectors of $U_{A_{r=64}}$.

This is measured using a normalized subspace similarity based on the Grassmann distance. The formula for this measure, denoted $\phi(A_{r=8}, A_{r=64}, i, j)$, is given as follows:

$$
\phi(A_{r=8}, A_{r=64}, i, j) = \frac{||U_{A_{r=8}}^{(i)} {U_{A_{r=64}}^{(j)}}^T||_F^2}{\min(i, j)}
$$

where $U_{A_{r}}^{(i)}$ represents the columns of $U_{A_{r}}$ corresponding to the top $i$ singular vectors, and $||\cdot||_F$ denotes the Frobenius norm.

The measure $\phi(·)$ ranges from 0 to 1, where 1 represents a complete overlap of the subspaces (i.e., they are the same), and 0 represents a complete separation (i.e., they are orthogonal). This is a normalized measure because it's divided by $\min(i, j)$, which is the maximum possible square of the Frobenius norm of the product matrix $U_{A_{r=8}}^{(i)} {U_{A_{r=64}}^{(j)}}^T$.

This process is performed for all pairs $(i, j)$ where $1 \leq i \leq 8$ and $1 \leq j \leq 64$. The results give an understanding of how much the learned subspaces for different ranks overlap with each other.

This can also be performed on two layers $\Delta W_1 = B_1A_1$ and $\Delta W_2 = B_2A_2$  in two different LoRAs. In particular, suppose we choose a layer `n` of each LoRA and run the subspace similarity measure comparison on $U_{\Delta W_1}^{(i)} {U_{\Delta W_2}^{(j)}}^T$. Then this will tell us how much those to LoRAs overlap with one another. 

This could be useful in determining which LoRAs to merge. If we run this analysis on all of the weight matrices of two different LoRAs, then we can determine how much layer `n` of `lora1` overlaps with layer `n` of `lora2`. If the overlap is small, then the two weight martices $\Delta W_1^{(n)} = B_1^{(n)}A_1^{(n)}$ and $\Delta W_2^{(n)} = B_2^{(n)}A_2^{(n)}$ may express very different things because the subspaces that they span do not overlap very much. So, to be more explicit, we compute

$$
\phi(\Delta W_1^{(n)}, \Delta W_2^{(n)}, i, j) = \frac{||U_{\Delta W_1^{(n)}}^{(i)} {U_{\Delta W_2^{(n)}}^{(j)}}^T||_F^2}{\min(i, j)}
$$

for a weight matrix $\Delta W_1^{(n)}$ from the first LoRA, and the corresponding $\Delta W_2^{(n)}$ from the second LoRA. This could indicate that merging the two LoRAs will create a more general model, able to create a wider range of diverse styles. This might also help in explaining why two LoRAs create something very muddy or undesirable when merges. Obviously, this is all conjecture based on a mathematical analysis that needs to be tested, and it does not provide a precise theshold for the overlap. What upper or lower bound might we use for this subspace similarity measure $\phi$? Could this hypthesis be wrong, or inverted? That is, is it possible that in some cases we actually want *high* overlap between models so that we merge very similar concepts?

In [18]:
import torch

# The A matrices
A1 = lora1_tensors['lora_te_text_model_encoder_layers_0_mlp_fc1.lora_down.weight'].float()
A2 = lora2_tensors['lora_te_text_model_encoder_layers_0_mlp_fc1.lora_down.weight'].float()

# The B matrices
B1 = lora1_tensors['lora_te_text_model_encoder_layers_0_mlp_fc1.lora_up.weight'].float()
B2 = lora2_tensors['lora_te_text_model_encoder_layers_0_mlp_fc1.lora_up.weight'].float()

# Compute the update matrices
Delta_W1 = torch.matmul(B1, A1)
Delta_W2 = torch.matmul(B2, A2)

# Compute the SVD of the update matrices
U1, _, _ = torch.svd(Delta_W1)
U2, _, _ = torch.svd(Delta_W2)

# Define the subspace similarity measure
def phi(U1, U2, i, j):
    U1_i = U1[:, :i]  # First i columns of U1
    U2_j = U2[:, :j]  # First j columns of U2
    
    product = torch.matmul(U1_i.t(), U2_j)  # Matrix multiplication
    norm = torch.norm(product)  # Frobenius norm
    
    return norm ** 2 / min(i, j)

# Calculate the subspace similarity measure
i = U1.size(1)  # Number of columns in U1
j = U2.size(1)  # Number of columns in U2
result = phi(U1, U2, i, j)  # Replace i and j with the desired values

print(f'The subspace similarity measure is: {result}')

The subspace similarity measure is: 0.26528483629226685


In [19]:
import torch

# Gather all keys and sort them
all_keys = sorted(list(lora1_tensors.keys()))

# Filter keys for lora_down and lora_up pairs
lora_down_keys = [key for key in all_keys if 'lora_down' in key]
lora_up_keys = [key for key in all_keys if 'lora_up' in key]

# Ensure we have matching pairs of keys
assert len(lora_down_keys) == len(lora_up_keys), "Mismatch in number of 'lora_down' and 'lora_up' keys"

# Define the subspace similarity measure
def phi(U1, U2, i, j):
    U1_i = U1[:, :i]  # First i columns of U1
    U2_j = U2[:, :j]  # First j columns of U2
    
    product = torch.matmul(U1_i.t(), U2_j)  # Matrix multiplication
    norm = torch.norm(product)  # Frobenius norm
    
    return norm ** 2 / min(i, j)

# Iterate over all layers
for layer in range(len(lora_down_keys)):
    try:
        # Extract the corresponding A and B matrices
        A1 = lora1_tensors[lora_down_keys[layer]].float()
        B1 = lora1_tensors[lora_up_keys[layer]].float()

        A2 = lora2_tensors[lora_down_keys[layer]].float()
        B2 = lora2_tensors[lora_up_keys[layer]].float()

        # Print the shapes of A1 and B1 matrices for troubleshooting
        print(f"A1 shape: {A1.shape}")
        print(f"B1 shape: {B1.shape}")

        # Compute the update matrices
        Delta_W1 = torch.matmul(B1, A1)
        Delta_W2 = torch.matmul(B2, A2)

        # Compute the SVD of the update matrices
        U1, _, _ = torch.svd(Delta_W1)
        U2, _, _ = torch.svd(Delta_W2)

        # Calculate the subspace similarity measure
        i = U1.size(1)  # Number of columns in U1
        j = U2.size(1)  # Number of columns in U2
        result = phi(U1, U2, i, j)  # Replace i and j with the desired values

        print(f"The subspace similarity measure for layer {layer} is: {result}")

    except RuntimeError as e:
        # Print the layer number and the error message
        print(f"Error occurred at layer {layer}: {e}")

A1 shape: torch.Size([32, 768])
B1 shape: torch.Size([3072, 32])
The subspace similarity measure for layer 0 is: 0.26528483629226685
A1 shape: torch.Size([32, 3072])
B1 shape: torch.Size([768, 32])
The subspace similarity measure for layer 1 is: 0.9999878406524658
A1 shape: torch.Size([32, 768])
B1 shape: torch.Size([768, 32])
The subspace similarity measure for layer 2 is: 0.9999885559082031
A1 shape: torch.Size([32, 768])
B1 shape: torch.Size([768, 32])
The subspace similarity measure for layer 3 is: 0.9999887347221375
A1 shape: torch.Size([32, 768])
B1 shape: torch.Size([768, 32])
The subspace similarity measure for layer 4 is: 0.9999908804893494
A1 shape: torch.Size([32, 768])
B1 shape: torch.Size([768, 32])
The subspace similarity measure for layer 5 is: 0.9999882578849792
A1 shape: torch.Size([32, 768])
B1 shape: torch.Size([3072, 32])
The subspace similarity measure for layer 6 is: 0.26571375131607056
A1 shape: torch.Size([32, 3072])
B1 shape: torch.Size([768, 32])
The subspace 

## Grassmannian (Fréchet/Karcher) Mean For Square Update Matrices in two LoRA Models

The following is supposed to implement the Fréchet (a.k.a. Karcher) mean of two points on the Grassmannian. If we know that the matrices are square, as most of the weight matrices in Stable Diffusion are (and therefore so are the update matrices of LoRA models), then we can use it to compute the Fréchet mean of any of the square matrices for two LoRAs. In particular, we can compute the Fréchet mean of two update weight matrices $\Delta W_1^{(n)}$ and $\Delta W_2^{(n)}$ for some fixed layer `n` of the LoRA models. 

In [20]:
import numpy as np
from scipy.linalg import svd

def grassmann_mean(points, max_iters=100, tol=1e-6):
    """
    Computes the Fréchet mean of a set of points on the Grassmann manifold.

    Parameters:
    - points: a list of numpy arrays, each representing a point on the Grassmannian.
    - max_iters: the maximum number of iterations for the algorithm.
    - tol: the tolerance for convergence.

    Returns:
    - The Fréchet mean as a numpy array.
    """
    # Initialize the mean to the first point
    mean = np.linalg.qr(points[0])[0]
    for _ in range(max_iters):
        # Compute the tangent space at the current mean
        tangent_space = sum([log_map(mean, np.linalg.qr(p)[0]) for p in points]) / len(points)
        # Update the mean
        new_mean = exp_map(mean, tangent_space)
        # Check for convergence
        if np.linalg.norm(new_mean - mean) < tol:
            return new_mean
        mean = new_mean
    return mean

def log_map(p, q):
    """
    Computes the logarithm map at p of q, i.e., the tangent vector at p that points towards q.

    Parameters:
    - p: a numpy array representing a point on the Grassmannian.
    - q: a numpy array representing a point on the Grassmannian.

    Returns:
    - The tangent vector at p that points towards q.
    """
    # Compute the SVD of p^T q
    u, s, vt = svd(np.dot(p.T, q))
    # Clip the singular values to the range [-1, 1]
    s = np.clip(s, -1, 1)
    # Compute the logarithm map
    return np.dot(p, np.dot(u, np.dot(np.diag(np.arccos(s)), vt)))


def exp_map(p, x):
    """
    Computes the exponential map at p of x, i.e., the point on the Grassmannian reached by following the geodesic in the direction of x from p.

    Parameters:
    - p: a numpy array representing a point on the Grassmannian.
    - x: a numpy array representing a tangent vector at p.

    Returns:
    - The point on the Grassmannian reached by following the geodesic in the direction of x from p.
    """
    # Compute the SVD of x
    u, s, vt = svd(x)
    # Compute the exponential map
    return np.dot(p, np.dot(u, np.dot(np.diag(np.cos(s)), vt))) + np.dot(u, np.dot(np.diag(np.sin(s)), vt))

In [21]:
import numpy as np

def generate_random_orthogonal_matrix(dim):
    """
    Generates a random orthogonal matrix of the given dimension.

    Parameters:
    - dim: the dimension of the matrix.

    Returns:
    - A numpy array representing an orthogonal matrix of the given dimension.
    """
    # Generate a random matrix
    mat = np.random.randn(dim, dim)
    # Compute the QR decomposition of the matrix
    q, _ = np.linalg.qr(mat)
    return q

# Set a random seed for reproducibility
np.random.seed(0)

# Generate a list of points on the Grassmannian
points = [generate_random_orthogonal_matrix(3) for _ in range(5)]

# Compute the Fréchet mean of the points
mean = grassmann_mean(points)

print(mean)

[[ 0.41942679  0.55888978 -0.84660875]
 [ 0.2804597  -0.98964856 -0.45203556]
 [-0.9643001   0.02577428 -0.5265871 ]]
